Treinando uma rede neural usando pytorch
========================================



## Introdução



O notebook anterior foi o *season finale* da construção da rede neural usando Python puro. Este notebook é o epílogo.

Aqui veremos como podemos construir e treinar uma rede neural usando `pytorch`, um módulo especializado em redes neurais artificiais.

Motivos para usarmos o `pytorch`:

1.  A `MLP` que criamos em Python puro não é otimizada. Tente treinar uma rede com uns 100 neurônios em uma camada e verá como ela é bem lenta! `pytorch` é escrito por diversos programadores que além de primar pelas contas estarem corretas, eles fazem de tudo para que elas ocorram de forma eficiente.

2.  `pytorch` já tem praticamente tudo que precisamos implementado! Quer calcular o gradiente local da função arco tangente? Ele já tem! Quer usar a função de ativação ReLU? Ele já tem!

3.  `pytorch` tem suporte a treinar uma rede neural usando processamento gráfico (GPU). Isso acelera muito o treino de redes neurais artificiais complexas. Temos GPUs para usarmos no HPC da Ilum!

4.  `pytorch` é usado tanto na academia quanto no mundo corporativo. Junto com `tensorflow` são os dois módulos estado da arte para o projeto e treino de redes neurais.



## Objetivo



Treinar uma rede neural artificial tipo Multilayer Perceptron usando `pytorch`.



## Importações



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

## Código e discussão



### Dados



Vamos treinar nossa rede neural com nosso velho amigo, o dataset de diamantes! Como todo processo de treino de aprendizado de máquina, precisamos reservar um conjunto de dados para treino e outro para teste.



In [2]:
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455
DATASET_NAME = "diamonds"
FEATURES = ["carat", "depth", "table", "x", "y", "z"]
TARGET = ["price"]

df = sns.load_dataset(DATASET_NAME)

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.reindex(FEATURES, axis=1)
y_treino = df_treino.reindex(TARGET, axis=1)
X_teste = df_teste.reindex(FEATURES, axis=1)
y_teste = df_teste.reindex(TARGET, axis=1)

Redes neurais costumam se dar bem quando os dados estão entre $[0, 1]$ ou entre $[-1, 1]$. Redes neurais costumam não se dar bem quando os dados estão em escalas muito diferentes. Por conta disso, normalizar os dados antes de prosseguir é recomendado! Vamos usar o `MinMaxScaler` do `scikit-learn` que aprendemos semestre passado.



In [3]:
normalizador_x = MinMaxScaler()
normalizador_y = MinMaxScaler()

normalizador_x.fit(X_treino)
normalizador_y.fit(y_treino)

X_treino = normalizador_x.transform(X_treino)
y_treino = normalizador_y.transform(y_treino)
X_teste = normalizador_x.transform(X_teste)
y_teste = normalizador_y.transform(y_teste)

### Tensores



Na nossa rede neural feita em Python puro, a base de tudo era a classe `Valor`. Para o `pytorch`, a base de tudo são os tensores! Tensores são objetos que armazenam dados de forma similar aos arrays de `numpy`, porém registram todas as operações realizadas para o cálculo do gradiente local (assim como nós fizemos com a classe `Valor`).



In [4]:
X_treino = torch.tensor(X_treino, dtype=torch.float32)
y_treino = torch.tensor(y_treino, dtype=torch.float32)
X_teste = torch.tensor(X_teste, dtype=torch.float32)
y_teste = torch.tensor(y_teste, dtype=torch.float32)

Vamos checar os dados para ver como estão.



In [5]:
print(X_treino)
print()
print(y_treino)

tensor([[0.0686, 0.5611, 0.2885, 0.4777, 0.0876, 0.1022],
        [0.0624, 0.5250, 0.2500, 0.4730, 0.0871, 0.0994],
        [0.0478, 0.4917, 0.2692, 0.4572, 0.0829, 0.0934],
        ...,
        [0.1185, 0.5333, 0.2692, 0.5410, 0.0995, 0.1142],
        [0.0166, 0.5139, 0.2885, 0.3929, 0.0708, 0.0811],
        [0.0333, 0.5361, 0.2692, 0.4209, 0.0776, 0.0890]])

tensor([[0.0676],
        [0.0586],
        [0.0434],
        ...,
        [0.1527],
        [0.0172],
        [0.0216]])


### Criando a rede neural



Para criar uma rede neural usando `pytorch` é necessário criar uma classe. Observe aqui que a classe criada é baseada na classe `nn.Module` do `pytorch`. Classes podem ser baseadas em outras classes! Trata-se da característica de herança das classes.

Observe que definimos as camadas da rede neural dentro de um objeto `nn.Sequential`.

Observe também o método `forward`. Este funciona de forma similar ao dunder `__call__` que vimos anteriormente. O `pytorch` requer que usemos o `forward` e não o `__call__`, então aceitamos e seguimos em frente.



In [6]:
class MLP(nn.Module):
    def __init__(
        self, num_dados_entrada, neuronios_c1, neuronios_c2, num_targets
    ):
        # Temos que inicializar a classe mãe
        super().__init__()

        # Definindo as camadas da rede
        self.camadas = nn.Sequential(
                    nn.Linear(num_dados_entrada, neuronios_c1),
                    nn.ReLU(),
                    nn.Linear(neuronios_c1, neuronios_c2),
                    nn.ReLU(),
                    nn.Linear(neuronios_c2, num_targets),
                )

    def forward(self, x):
        """Esse é o método que executa a rede do pytorch."""
        x = self.camadas(x)
        return x

Agora podemos criar a nossa rede neural!



In [7]:
NUM_DADOS_DE_ENTRADA = X_treino.shape[1]
NUM_DADOS_DE_SAIDA = y_treino.shape[1]
NEURONIOS_C1 = 50
NEURONIOS_C2 = 20

minha_MLP = MLP(NUM_DADOS_DE_ENTRADA, NEURONIOS_C1, NEURONIOS_C2, NUM_DADOS_DE_SAIDA)

E podemos checar os parâmetros internos dela!



In [8]:
for p in minha_MLP.parameters():
    print(p)

Parameter containing:
tensor([[ 0.0199, -0.1936,  0.0561, -0.0692, -0.3076, -0.3728],
        [-0.1123,  0.3368,  0.1683,  0.0313,  0.2024, -0.0668],
        [ 0.0324, -0.3962,  0.0161, -0.4014,  0.1281, -0.0215],
        [ 0.2519,  0.0495,  0.0115,  0.0885,  0.3909,  0.0862],
        [-0.3579, -0.2132, -0.1828,  0.0731,  0.3085, -0.0839],
        [-0.2661, -0.0241,  0.1547, -0.1518,  0.3124, -0.0505],
        [ 0.2042,  0.2931, -0.3882, -0.3374,  0.0148, -0.3324],
        [-0.2276,  0.1957,  0.2728, -0.3128,  0.2514,  0.1276],
        [-0.3091, -0.0503, -0.0476, -0.0715, -0.3332, -0.1593],
        [ 0.1300,  0.3454,  0.2958,  0.3107, -0.2222,  0.3502],
        [ 0.1085, -0.1378, -0.2984,  0.2321,  0.0844, -0.1032],
        [-0.0108,  0.2358, -0.2246,  0.0658,  0.0029,  0.3105],
        [-0.0452, -0.0943,  0.1095, -0.1947,  0.0759, -0.1161],
        [ 0.0143, -0.3984, -0.3926, -0.2825,  0.2073,  0.1998],
        [-0.0886,  0.2711,  0.2053,  0.2480, -0.2911, -0.3523],
        [ 0.2909, 

Também podemos realizar uma previsão, mas ela provavelmente será bem ruim!



In [9]:
y_prev = minha_MLP(X_treino)
y_prev

tensor([[-0.0656],
        [-0.0633],
        [-0.0622],
        ...,
        [-0.0679],
        [-0.0577],
        [-0.0607]], grad_fn=<AddmmBackward>)

### A função de perda e o otimizador



O `pytorch` já vem com diversas funções de perda já implementadas. A que computa o erro quadrático médio, por exemplo, já está pronta!

Fora isso, precisamos definir o nosso otimizador. O otimizador é quem cuida de atualizar os parâmetros da nossa rede. Nós implementamos na nossa rede em Python puro a descida do gradiente. Aqui usaremos um otimizador que é uma modificação da descida do gradiente, chamado `Adam`. Trata-se, simplificadamente, de uma descida do gradiente com taxa de aprendizado individualizada para cada parâmetro e que se altera ao longo do aprendizado. `Adam` é um poderosíssimo otimizador! Não tem porque não usarmos ele neste caso.



In [10]:
TAXA_DE_APRENDIZADO = 0.001

# função perda será o erro quadrático médio
fn_perda = nn.MSELoss()

# otimizador será o Adam, um tipo de descida do gradiente
otimizador = optim.Adam(minha_MLP.parameters(), lr=TAXA_DE_APRENDIZADO)

### O treino da rede



Para avisar o `pytorch` que iremos treinar a rede, vamos invocar o método `train` da nossa rede. Toda rede recém-criada já está no modo treino, mas aqui deixarei explícito para ficar bem claro.



In [11]:
minha_MLP.train()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Observe que o ciclo do treino tem as mesmas etapas que vimos no notebook anterior, nada de novo! Aqui basta observar como executamos essas etapas no `pytorch`.



In [12]:
NUM_EPOCAS = 1000

y_true = y_treino

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = minha_MLP(X_treino)

    # zero grad
    otimizador.zero_grad()

    # loss
    loss = fn_perda(y_pred, y_true)

    # backpropagation
    loss.backward()

    # atualiza parâmetros
    otimizador.step()

    # mostra resultado
    print(epoca, loss.data)

0 tensor(0.1180)
1 tensor(0.1126)
2 tensor(0.1076)
3 tensor(0.1027)
4 tensor(0.0981)
5 tensor(0.0937)
6 tensor(0.0898)
7 tensor(0.0861)
8 tensor(0.0825)
9 tensor(0.0791)
10 tensor(0.0757)
11 tensor(0.0728)
12 tensor(0.0702)
13 tensor(0.0678)
14 tensor(0.0657)
15 tensor(0.0637)
16 tensor(0.0617)
17 tensor(0.0598)
18 tensor(0.0580)
19 tensor(0.0563)
20 tensor(0.0547)
21 tensor(0.0532)
22 tensor(0.0518)
23 tensor(0.0504)
24 tensor(0.0492)
25 tensor(0.0481)
26 tensor(0.0470)
27 tensor(0.0460)
28 tensor(0.0450)
29 tensor(0.0441)
30 tensor(0.0433)
31 tensor(0.0426)
32 tensor(0.0420)
33 tensor(0.0416)
34 tensor(0.0412)
35 tensor(0.0409)
36 tensor(0.0407)
37 tensor(0.0406)
38 tensor(0.0405)
39 tensor(0.0404)
40 tensor(0.0403)
41 tensor(0.0403)
42 tensor(0.0402)
43 tensor(0.0401)
44 tensor(0.0400)
45 tensor(0.0399)
46 tensor(0.0397)
47 tensor(0.0396)
48 tensor(0.0394)
49 tensor(0.0391)
50 tensor(0.0389)
51 tensor(0.0386)
52 tensor(0.0384)
53 tensor(0.0381)
54 tensor(0.0378)
55 tensor(0.0376)
56

Após o treino, podemos checar a performance da nossa rede. Observe a linha que inicia com `with`. Tudo que estiver no bloco do `with torch.no_grad()` é computado normalmente, porém o grafo computacional e os gradientes locais não são computados. Use isso sempre que for realizar contas com tensores fora do treino!!



In [13]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_treino)
    y_pred = minha_MLP(X_treino)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[1577.99996901] [1423.98612261]
[1409.99999118] [1516.69741058]
[1128.99997044] [1214.37226534]
[12991.99967575] [11928.08711624]
[14720.0001564] [14491.65571594]
[2938.99994564] [2701.16921997]
[6635.00014687] [7935.22563744]
[11215.99973297] [9302.07644081]
[1040.99999189] [1357.93049955]
[5040.00003624] [5223.63857651]
[9541.99988747] [8720.98738289]
[3792.99988461] [5292.52721405]
[1002.0000155] [777.6214577]
[2542.00005341] [1411.69824266]
[6788.00001717] [6557.78582573]
[9386.00025749] [8675.91767502]
[1245.99996853] [1717.25521278]
[997.00000262] [665.54361212]
[708.00000262] [753.75360692]
[1435.99997544] [1393.99321127]
[2590.00006676] [2480.52647781]
[1392.99997497] [1613.98302889]
[11755.99946976] [9453.65635872]
[2007.99997377] [1248.7653203]
[2707.99998474] [2592.09623289]
[631.00000417] [673.15504754]
[652.99999881] [988.83093548]
[2492.00006247] [2724.41487694]
[594.99999416] [555.81714928]
[760.0000056] [1090.6976366]
[1179.00003028] [1095.2850585]
[1789.9999361] [1715.

### O teste da rede



Vamos supor que você já testou diversas arquiteturas de rede e está pronto para usar os dados de teste. Nesta etapa precisamos indicar para o `pytorch` que o treino acabou e estamos na etapa de avaliação. Fazemos isso rodando o método `eval`.



In [14]:
minha_MLP.eval()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Agora podemos realizar o teste!



In [15]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_teste)
    y_pred = minha_MLP(X_teste)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[523.00000858] [930.61922646]
[628.00000334] [686.98359787]
[442.00000328] [752.934075]
[2284.99997044] [2683.41981506]
[1040.99999189] [845.31490433]
[809.99999654] [1350.72255993]
[3049.99987316] [3454.27068233]
[720.00000596] [667.18887722]
[877.99999249] [666.48964989]
[1292.00000429] [1566.98679399]
[7056.00004578] [4934.0768795]
[4176.99999142] [5895.33118057]
[13701.00020599] [14905.27087021]
[2948.0000515] [4097.74338627]
[16732.99985504] [14267.27898407]
[1815.00000048] [2052.86517763]
[16220.00016022] [12212.29808426]
[5967.00019073] [4682.66234684]
[473.00000042] [651.66710532]
[1078.9999795] [1097.66414022]
[452.00000319] [629.0328933]
[801.99998283] [832.91774428]
[2458.99994993] [2565.69261503]
[2492.99998236] [2754.02027798]
[582.00000203] [696.24401915]
[1405.9999671] [1386.07584667]
[8715.99991035] [11614.24125671]
[7634.00004578] [6503.92354774]
[506.99999839] [641.34185755]
[665.99999094] [751.6880337]
[1815.00000048] [1568.65369463]
[707.00001383] [634.18917811]
[59

E, finalmente, computar alguma métrica para medir a performance do nosso modelo.



In [16]:
RMSE = mean_squared_error(y_true, y_pred, squared=False)
print(f'Loss do teste: {RMSE}')

Loss do teste: 1379.008681291278


## Conclusão

<p style="text-align: justify">Ao longo do trabalho, foram abordados diversos aspectos fundamentais do uso do PyTorch para treinamento de redes neurais. Desde a preparação dos dados, com a separação em conjuntos de treinamento e teste, até a criação da arquitetura da rede neural, foram exploradas as funcionalidades do PyTorch de forma clara e didática.</p>

<p style="text-align: justify">Uma das grandes vantagens do PyTorch é a sua eficiência em termos de desempenho. Ao utilizar técnicas de otimização e processamento gráfico, o PyTorch permite treinar redes neurais complexas de forma rápida e eficiente. Isso se deve à sua implementação cuidadosa e à utilização de algoritmos avançados, como o otimizador Adam, que se adaptam ao longo do treinamento para buscar os melhores resultados. Além disso, o PyTorch oferece uma grande variedade de funções e camadas já implementadas, o que facilita a construção da arquitetura da rede neural.</p>

<p style="text-align: justify">O trabalho também apresentou a etapa de teste da rede neural, utilizando os dados de teste separados anteriormente. Através do método eval, o PyTorch foi configurado para a fase de avaliação, e as previsões da rede foram comparadas com os valores reais para calcular a métrica de desempenho, como o erro quadrático médio (RMSE).</p>

<p style="text-align: justify">À medida que avançamos nessa disciplina, entendemos o passo-a-passo de como construir uma rede neural e, agora, com os recursos do PyTorch para criar modelos mais complexos, ajustar hiperparâmetros, realizar validação cruzada e explorar técnicas de regularização e otimização, vemos o quão aplamente isso pode ser aplicado, não necessariamente de uma forma muito trabalhosa. Desta forma, O PyTorch nos fornece uma base sólida para a construção de modelos de aprendizado de máquina mais avançados e eficientes, de forma mais prática.</p>